### Tbot - Trade Bot

 Tbot is a Robot of trade developed to trade cryptourrencies or s&p500 stocks, based on https/json queries. With few adjustments it may help traders to improve their workflow. In this file we shall compile three steps followed at the "Algorithimic Trading Using Python" course created by the api developer and completed by the developer of this api. ("https://freecodecamp.org")
 
### Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this api. 

In [53]:
import numpy as np
import pandas as pd
import requests
import secrets
import xlsxwriter
import math
import hmac
import time
import datetime
import hashlib
import binascii

### Timestamp data
Acquires timestamp and stores it

In [11]:
#Moved to API section because of query purposes

### API token, secret key and acess keys

API tokens (and other sensitive information) shall be stored in a secrets.py file that doesn't get pushed to your local Git repository. The AcessKey and SecretKey are provided by the server admin.

In [54]:
from secrets import SecretKey
from secrets import AccessKey

In [43]:
#signature encrypt
def _generate_signature(data):
    key = 'key'
    key_bytes= bytes(key , 'utf-8') 
    data_bytes = bytes(data, 'utf-8')
    return hmac.new(key_bytes, data_bytes , hashlib.sha256).hexdigest()

queryStr = "name=user&cpf=012345678912&birthday=1999-01-31"

#urlencode = "https://api.novadax.com/"
urlencode = "https://api.novadax.com/v1/orders/list"

timestamp_url = f'https://api.novadax.com/v1/common/timestamp'
Timestamp = requests.get(timestamp_url).json()
tsp = str(Timestamp['data'])
#print(timestamp)


qStr = f'{urlencode}{queryStr}{tsp}'
signStr = urlencode

#encode
signature = f'{qStr}{SecretKey}'
hmac_sha256 = _generate_signature(signature)
Sign = hmac_sha256
print(Sign)

#{método de requisição}\n{caminho de requisição}\n{string a ser codificada}\n{timeStamp}
signStrget = requests.get(signStr, headers={"X-Nova-Access-Key":AccessKey, "X-Nova-Signature": Sign, "X-Nova-Timestamp": tsp}).json()
print(signStrget)

d78289fade81913f81cc26bcaa688c143388fb8da4fc1ab8d4acfe57721a7890
{'code': 'A10003', 'data': None, 'message': "Authentication failed, The correct signature after encryption HMAC-SHA256 should be initiated with '8038a4'. Please check If you applied the HMAC-SHA256 hash function correctly; If the SecretKey is correctly input; If the signature string is 'GET\n/v1/orders/list\n\n1613419483413'."}


### Importing tht List of currencies, making the API Call and the table Dataframe
Now it's time to structure our API calls to the api url.
The next thing we need to do is import the constituents of the cryptocurrencies table.

In [55]:
#API call
symbols = ['ETH_USDT', 'BTC_USDT']
symbol = 'ETH_USDT'
api_url = f'https://api.novadax.com/v1/market/ticker?symbol={symbol}'
df = requests.get(api_url).json()

In [56]:
#Parse for data then get price
dfdata = df['data']
price = float(dfdata['lastPrice'])
baseVolume24h = dfdata['baseVolume24h']

In [46]:
currency = 'R$'
def position():
    global position
    position = input("Enter the value of your position in BRL:")

    try:
        val = float(position)
    except ValueError:
        print("That's not a number! \n Try again:")
        position = input("Enter the value of your position in BRL:")

        
position()
print(currency, position)

Enter the value of your position in BRL:885
R$ 885


In [47]:
def dolar_day():
    global dolar_day
    dolar_day = input("Enter the dolar value today in BRL:")

    try:
        val = float(dolar_day)
    except ValueError:
        print("That's not a number! \n Try again:")
        dolar_day = input("Enter the dolar value today in BRL:")
dolar_day()
print(currency, dolar_day)

Enter the dolar value today in BRL:5.36
R$ 5.36


In [48]:
#Final Dataframe
my_columns = ['Symbol', 'Currency Price', 'Base Volume 24h', 'Number of Shares to Buy']
f_dataframe = pd.DataFrame(columns = my_columns)

for symbol in symbols:
    api_url = f'https://api.novadax.com/v1/market/ticker?symbol={symbol}'
    fddata = requests.get(api_url).json()
    fdata = fddata['data']
    price = float(fdata['lastPrice'])
    baseVolume24h = fdata['baseVolume24h']
    shares_to_buy = (float(position)/float(dolar_day)/float(price))
    for i in fdata['symbol'][:1]:
        f_dataframe.append(
            pd.Series(
                [
                    symbol,
                    price,
                    baseVolume24h,
                    'N/A'
                ]
                ,index = my_columns),
            ignore_index=True)
    
    
        f_dataframe = f_dataframe.append(
                                            pd.Series([symbol, 
                                                       fdata['lastPrice'], 

                                                       fdata['baseVolume24h'], 
                                                       shares_to_buy], 
                                                      index = my_columns), 
                                            ignore_index = True)

f_dataframe

,Symbol,Currency Price,Base Volume 24h,Number of Shares to Buy
0,ETH_USDT,1828.33,13.6332,0.090308
1,BTC_USDT,48614.28,2.510448,0.003396


### Sending the requisition to the web
The content in the order requisition should be sent to the server following the instructions given by the company or individual that holds the coin.

In [78]:
from novadax import RequestClient as NovaClient

nova_client = NovaClient(AccessKey, SecretKey)

result = nova_client.get_symbol(symbol)
print(result)

{'code': 'A10000', 'data': {'amountPrecision': 4, 'baseCurrency': 'ETH', 'minOrderAmount': '0.002', 'minOrderValue': '5', 'pricePrecision': 2, 'quoteCurrency': 'USDT', 'status': 'ONLINE', 'symbol': 'ETH_USDT', 'valuePrecision': 6}, 'message': 'Success'}


In [82]:
#orders
listtrades = nova_client.list_trades(symbol, limit=10)
print(listtrades)

{'code': 'A10000', 'data': [{'amount': '0.009027', 'price': '48739.19', 'side': 'BUY', 'timestamp': 1613419646712}, {'amount': '0.001646', 'price': '48614.28', 'side': 'SELL', 'timestamp': 1613419105662}, {'amount': '0.000568', 'price': '48668.72', 'side': 'SELL', 'timestamp': 1613418932777}, {'amount': '0.037266', 'price': '48644', 'side': 'BUY', 'timestamp': 1613418687545}, {'amount': '0.011059', 'price': '48640', 'side': 'BUY', 'timestamp': 1613418687545}, {'amount': '0.053986', 'price': '48594.58', 'side': 'BUY', 'timestamp': 1613418339623}, {'amount': '0.037303', 'price': '48497.74', 'side': 'SELL', 'timestamp': 1613417660447}, {'amount': '0.003247', 'price': '48499.19', 'side': 'BUY', 'timestamp': 1613415414771}, {'amount': '0.000464', 'price': '48428.59', 'side': 'SELL', 'timestamp': 1613414562185}, {'amount': '0.019642', 'price': '48543.93', 'side': 'SELL', 'timestamp': 1613411869722}], 'message': 'Success'}


In [84]:
my_columns = ['Symbol', 'Ammount','Price']
f_dataframe = pd.DataFrame(columns = my_columns)

for symbol in symbols:
    listtrades = nova_client.list_trades(symbol, limit=10)
    
    ldata = listtrades['data']
    Timestamp = (ldata['timestamp'])
    tsp = Timestamp
    amount = float(ldata['amount'])
    price = float(ldata['price'])
    
    for i in fdata['symbol'][:1]:
        f_dataframe.append(
            pd.Series(
                [
                    symbol,
                    amount,
                    price,
                ]
                ,index = my_columns),
            ignore_index=True)
    
    
        f_dataframe = f_dataframe.append(
            pd.Series([
                    symbol,
                    amount,
                    price,
                      ]
            ,index = my_columns), 
            ignore_index = True)

f_dataframe

TypeError: list indices must be integers or slices, not str